In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.ensemble import RandomForestClassifier

## Data Prep

In [3]:
# train_X = pd.read_csv('music_data/train_X.csv')
train_X = pd.read_csv('music_data/train_X_date_converted.csv')
train_y = pd.read_csv('music_data/train_Y.csv')

# val_X = pd.read_csv('music_data/valid_X.csv')
val_X = pd.read_csv('music_data/valid_X_date_converted.csv')
val_y = pd.read_csv('music_data/valid_Y.csv')

# test_X = pd.read_csv('music_data/test_X.csv')
test_X = pd.read_csv('music_data/test_X_date_converted.csv')
test_y = pd.read_csv('music_data/test_Y.csv')

# train_tiny_X = pd.read_csv('music_data/train_tiny_X.csv')
train_tiny_X = pd.read_csv('music_data/train_tiny_X_date_converted.csv')
train_tiny_y = pd.read_csv('music_data/train_tiny_Y.csv')

# val_tiny_X = val_X[:1000].copy()
val_tiny_X = pd.read_csv('music_data/val_tiny_X_date_converted.csv')
val_tiny_y = val_y[:1000].copy()

## Random Forest

In [9]:
catCols = ['msno', 'song_id', 'source_screen_name', 'source_system_tab', 'source_type', 'genre_ids', 'artist_name', 
           'composer', 'lyricist', 'language', 'city', 'gender', 'registered_via']
numCols = ['bd', 'song_length']

In [12]:
data_scaler = StandardScaler()
train_num = data_scaler.fit_transform(train_tiny_X[numCols])
val_num = data_scaler.transform(val_tiny_X[numCols])
# train_num_df = pd.DataFrame(train_num,columns = numCols)[num_select2]
# train_num_df = train_num_df.reset_index(drop=True)
# val_num_df = pd.DataFrame(val_num,columns = numCols)[num_select2]
# val_num_df = val_num_df.reset_index(drop=True)

train_cat = train_tiny_X[catCols]
one_hot = OneHotEncoder(handle_unknown='ignore')
one_hot.fit(train_cat)
feature_names = [str(i) for i in list(one_hot.get_feature_names())]
for i in range(4):
    if i == 0:
        train_piece = one_hot.transform(train_cat[i*10000:(i+1)*10000]).toarray()
        val_piece = one_hot.transform(val_tiny_X[catCols]).toarray()
    elif i == 3:
        train_piece = np.vstack((train_piece,one_hot.transform(train_cat[30000:]).toarray()))      
    else:
        train_piece = np.vstack((train_piece,one_hot.transform(train_cat[i*10000:(i+1)*10000]).toarray()))

# train_cat_df = pd.DataFrame(train_piece,columns = cat_select2)
# val_cat_df = pd.DataFrame(val_piece,columns = cat_select2)
# test_cat_df = pd.DataFrame(test_piece,columns = cat_select2)
# newcatCols = list(train_cat_df.columns)

# DNNtrain_X = pd.concat([train_num_df,train_cat_df],axis = 1)
# DNNval_X = pd.concat([val_num_df,val_cat_df],axis = 1)
# DNNtest_X = pd.concat([test_num_df,test_cat_df],axis = 1)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  This is separate from the ipykernel package so we can avoid doing imports until


ValueError: Found array with 0 sample(s) (shape=(0, 13)) while a minimum of 1 is required.

In [13]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 20, stop = 500, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
rf_random = RandomizedSearchCV(estimator = RandomForestClassifier(random_state=0), param_distributions = random_grid, n_iter = 20, cv = 3, verbose=1, random_state=0)
rf_random.fit(train_tiny_X, train_tiny_y)

Fitting 3 folds for each of 20 candidates, totalling 60 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:542: FutureWarning: From version 0.22, errors during fit will result in a cross validation score of NaN by default. Use error_score='raise' if you want an exception raised or error_score=np.nan to adopt the behavior from version 0.22.
  FutureWarning)


MemoryError: could not allocate 873725952 bytes

In [ ]:
print(rf_random.best_params_)

In [ ]:
clf = RandomForestClassifier(n_estimators=500,random_state=0,min_samples_split=10)
clf.fit(train_tiny_X, train_tiny_y)
print(classification_report(val_tiny_y, clf.predict(val_tiny_X)>0.5))
print(roc_auc_score(val_tiny_y, clf.predict(val_tiny_X)))